In [1]:
#Import packages
import pandas as pd

In [2]:
#global values

#COE file path
COE_fp="Relevant_datasets/M11-coe_results.csv"

#consumer price index file path
CPI_fp='Relevant_datasets/CPI.csv'

#Export output fp
COE_op='Relevant_datasets/COE_processed.csv'

In [3]:
#Import COE
COE=pd.read_csv(COE_fp)

#Import CPI
CPI=pd.read_csv(CPI_fp,header=8,usecols=['Data Series','All Items (Index)'])


In [4]:
#Clean CPI dataset

#rename columns
CPI.rename(columns={'All Items (Index)':'Index','Data Series':'Year_Quarter'},inplace=True)

#convert year Qtr column to string and remove white spaces
CPI.Year_Quarter.astype('str')
CPI['Year_Quarter'] = CPI.apply(lambda x: x.Year_Quarter.strip(),axis=1)

In [5]:
##Since 2024 Q2 data is not available, i decided to do a projected value

#Get Index value of each quarter
I_Q1_2024=CPI['Index'].loc[CPI['Year_Quarter']=='2024 1Q'].values[0]
I_Q4_2023=CPI['Index'].loc[CPI['Year_Quarter']=='2023 4Q'].values[0]
I_Q3_2023=CPI['Index'].loc[CPI['Year_Quarter']=='2023 3Q'].values[0]
I_Q2_2023=CPI['Index'].loc[CPI['Year_Quarter']=='2023 2Q'].values[0]

#Get each Qtr's % change from prev Quarter
Q1_2024=(I_Q1_2024-I_Q4_2023)/I_Q4_2023
Q4_2023=(I_Q4_2023-I_Q3_2023)/I_Q3_2023
Q3_2023=(I_Q3_2023-I_Q2_2023)/I_Q2_2023
print("sanity check Q1_2024 % change:",Q1_2024)
#Get average % change
AVG_Delta=(Q4_2023+Q1_2024+Q3_2023)/3
print("sanity check AVG_Delta:",AVG_Delta)
#Get projected index value for 2024 Q2
Proj_Val=I_Q1_2024*(1+AVG_Delta)
#Create a new row and append to dataset
new_row = pd.DataFrame({'Index':[int(Proj_Val)],'Year_Quarter':['2024 2Q']})
CPI = CPI.append(new_row, ignore_index=True)

sanity check Q1_2024 % change: 0.0017376194613379918
sanity check AVG_Delta: 0.006448145473993555


In [6]:
#Normalize using 2024Q2 as index base
index_base=CPI['Index'].loc[CPI['Year_Quarter']=='2024 2Q'].values[0]
#divide every year's index base by 2024Q2's index base, multiply by 100
CPI['Index_2024Q2']=(CPI['Index']/index_base)*100

#drop old index and rename the new index_2024Q2
CPI.drop(columns='Index',inplace=True)
CPI.rename(columns={'Index_2024Q2':'Index'},inplace=True)

In [7]:
##Parse Year and Quarter from "month" and create a Yr_Qtr column for easy merging
COE['month'] = pd.to_datetime(COE['month'])
COE['Year'] = COE['month'].dt.year
COE['Quarter'] = COE['month'].dt.quarter

COE['Year_Quarter']=COE.apply(lambda x: str(x['Year'])+" "+str(x['Quarter'])+str("Q") ,axis=1)

#Groupby Yr_Qtr,vehicle_class and average by premium (reset index because of row merging issue during groupby)
COE_grpby=COE.groupby(by=['vehicle_class','Year_Quarter',]).mean('premium').reset_index()

#keep only Yr_Qtr,vehicle_class,premium
COE_grpby=COE_grpby[['Year_Quarter','premium','vehicle_class','bids_success']]


#Consumer vehicles COE tend to be Vehicle classes A,B and D
COE_grpby=COE_grpby.loc[(COE_grpby['vehicle_class']=='Category A')|(COE_grpby['vehicle_class']=='Category B')|(COE_grpby['vehicle_class']=='Category D')]

In [8]:
#To calculate the preimum for the quarter, to give the mean of premium,
#weighted by the proportion of bid_success for each vehivle class

#Create a df of total bid success(tbs) by Yr_Qtr
COE_tbs=COE_grpby.groupby(['Year_Quarter']).sum('bids_success')
COE_tbs=COE_tbs['bids_success']

#merge to COE_grp_by
COE_grpby=COE_grpby.merge(COE_tbs,on='Year_Quarter',how='left')

#divide by bids success in COE_grp by to get weight for each Yr_Qtr
COE_grpby['weight']=COE_grpby['bids_success_x']/COE_grpby['bids_success_y']

#multiply by premium and groupby.sum(premium)
COE_grpby['premium_wt']=COE_grpby['weight']*COE_grpby['premium']
COE_processed=COE_grpby.groupby(['Year_Quarter']).sum().reset_index()

#keep only Yr_Qtr and premium_wt
COE_processed=COE_processed[['Year_Quarter','premium_wt']]

In [9]:
#visual check
COE_processed.head()

,Year_Quarter,premium_wt
0,2010 1Q,19189.190792
1,2010 2Q,27263.581898
2,2010 3Q,28267.604568
3,2010 4Q,34662.570729
4,2011 1Q,37754.461506


In [10]:
#Calculate real COE price (accounted for inflation using CPI)
COE_processed=COE_processed.merge(CPI,on='Year_Quarter',how='left')

#Generate real COE price= premium_wt*100/Index
COE_processed['Real_Premium']=COE_processed['premium_wt']*100/COE_processed['Index']


In [11]:
COE_processed

,Year_Quarter,premium_wt,Index,Real_Premium
0,2010 1Q,19189.190792,74.051724,25913.226215
1,2010 2Q,27263.581898,75.000000,36351.442531
2,2010 3Q,28267.604568,75.862069,37261.842386
3,2010 4Q,34662.570729,76.551724,45279.934735
4,2011 1Q,37754.461506,77.931034,48445.990428
5,2011 2Q,39501.234054,78.534483,50297.948960
6,2011 3Q,42156.019460,80.000000,52695.024325
7,2011 4Q,43032.217265,80.775862,53273.609421
8,2012 1Q,43401.477562,81.724138,53107.293219
9,2012 2Q,49793.597493,82.672414,60230.003224


In [12]:
#export 
COE_processed.to_csv(COE_op)